<a href="https://colab.research.google.com/github/Harshita-2205/Twitter-Sentiments-Analysis-Using-NLP/blob/main/Twitter_sentiments_Analysis_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!imkdir -p ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

/bin/bash: line 1: imkdir: command not found
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140



Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 91% 74.0M/80.9M [00:03<00:00, 43.4MB/s]
100% 80.9M/80.9M [00:03<00:00, 27.9MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('congo dataset extracted :)')

congo dataset extracted :)


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

data processing


In [ ]:
#data->csv file==>> dataframe
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

#NO. of rows and cols
data.shape

(1599999, 6)

In [ ]:
data.head()

#reading dataset and renaming cols
cols = ['target','id','date','flag','user','text']
data.columns = cols
data.head()
data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#checking distribution of target col
data['target'].value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

In [ ]:
data.replace({'target':{4:1}},inplace=True)
data['target'].value_counts()
#0->negative tweet
#1->positive tweet

target
1    800000
0    799999
Name: count, dtype: int64

stemming


In [ ]:
port_stem = PorterStemmer()
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
data['stemmed_content'] = data['text'].apply(stemming)

In [ ]:
data.head()
print(data['stemmed_content'])

0          upset updat facebook text might cri result sch...
1          kenichan dive mani time ball manag save rest g...
2                            whole bodi feel itchi like fire
3                              nationwideclass behav mad see
4                                        kwesidei whole crew
                                 ...                        
1599994                           woke school best feel ever
1599995    thewdb com cool hear old walt interview http b...
1599996                         readi mojo makeov ask detail
1599997    happi th birthday boo alll time tupac amaru sh...
1599998    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1599999, dtype: object


In [ ]:
print(data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599994    1
1599995    1
1599996    1
1599997    1
1599998    1
Name: target, Length: 1599999, dtype: int64


In [ ]:
#separate label and data
x = data['stemmed_content'].values
y = data['target'].values
print (x)
print (y)

[0 0 0 ... 1 1 1]


split data into train test

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)
print(x.shape,x_train.shape,x_test.shape)

(1599999,) (1279999,) (320000,)


convert text->numerical data

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)

x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)

logistic regression


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [ ]:
#check accuracy of train data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)
print('accuracy on training data : ',training_data_accuracy)

#check accuracy of test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)
print('accuracy on test data : ',test_data_accuracy)
#

accuracy on training data :  0.8102912580400453
accuracy on test data :  0.777909375


In [ ]:
import pickle
filename = 'sentiment_analysis_model.sav'
pickle.dump(model,open(filename,'wb'))

checking model for future predictions

In [ ]:
tweet  = x_test[501]
print(x_test[501])
print(y_test[501])
predict = model.predict(tweet)
print(predict)

if predict[0]==0:
  print('negative tweet')
else:
  print('positive tweet')

  (0, 410164)	0.3076248633122285
  (0, 400876)	0.34000167092974176
  (0, 376842)	0.2831873119852836
  (0, 376221)	0.30531158402767467
  (0, 299246)	0.26086686001035564
  (0, 288287)	0.30536667961527386
  (0, 286650)	0.25676445916696744
  (0, 158892)	0.3650310000966798
  (0, 150812)	0.2156617615744143
  (0, 133726)	0.4575692828709015
0
[0]
negative tweet
